#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [49]:
import pandas as pd
import pymongo

In [50]:
client = pymongo.MongoClient("mongo")
db_series = client.series

In [51]:
collection_kickstarter = db_series["kickstarter"]
collection_kickstarter.delete_many({})

In [52]:
print(client.list_database_names())

['admin', 'config', 'local', 'series']


In [53]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv", index_col="ID")
df_ks = df_ks.rename(columns={"ID":"_id"})
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
ID,,,,,,,,,,,,,
872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [54]:
df_ks.columns[[6,8,10,12]]

Index(['launched', 'state', 'country', 'usd_pledged_real'], dtype='object')

## Question 0

In [55]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 872782264 to 891117261
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              149998 non-null  object 
 1   category          150000 non-null  object 
 2   main_category     150000 non-null  object 
 3   currency          150000 non-null  object 
 4   deadline          150000 non-null  object 
 5   goal              150000 non-null  object 
 6   launched          150000 non-null  object 
 7   pledged           150000 non-null  object 
 8   state             150000 non-null  object 
 9   backers           150000 non-null  object 
 10  country           150000 non-null  object 
 11  usd pledged       148518 non-null  object 
 12  usd_pledged_real  150000 non-null  float64
dtypes: float64(1), object(12)
memory usage: 16.0+ MB


### Netoyer les données

In [56]:
def what_types(df, column):
    types = set()
    df[column].apply(lambda x: types.add(type(x)))
    return types

In [57]:
def to_another_type(value, ptype):
    if not isinstance(value, ptype):
        return None
    return value

In [58]:
# 'launched' & 'deadline' columns

def to_datetime(value):
    try:
        return pd.to_datetime(value)
    except:
        return None
    
columns = ["launched", "deadline"]
for col in columns:
    df_ks[col] = df_ks[col].apply(to_datetime)


In [59]:
# 'state' & 'country' columns

columns = ["state", "country"]
for col in columns:
    df_ks[col] = df_ks[col].apply(lambda x:to_another_type(x, str)).astype("category")

In [60]:
# 'usd_pledged_real' column

df_ks['usd_pledged_real'] = df_ks['usd_pledged_real'].astype("float64")

In [61]:
# integers/float type columns

columns = ["goal", "pledged", "backers", "usd pledged"]
for col in columns:
    print("in {} : {}".format(col, what_types(df_ks, col)))

in goal : {<class 'str'>, <class 'float'>}
in pledged : {<class 'str'>, <class 'float'>}
in backers : {<class 'str'>, <class 'int'>}
in usd pledged : {<class 'str'>, <class 'float'>}


In [62]:
# float type columns

columns = ["goal", "pledged", "usd pledged"]

for col in columns:
    df_ks[col] = df_ks[col].apply(lambda x: to_another_type(x, float))
    df_ks[col] = df_ks[col].fillna(0.0)
    df_ks[col] = df_ks[col].astype("float64")

In [63]:
def change_backers(value):
    if isinstance(value, str):
        try:
            return int(value)
        except:
            return 0
    return value

df_ks["backers"] = df_ks["backers"].apply(change_backers)
df_ks["backers"] = df_ks["backers"].astype("int64")

In [64]:
df_ks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 872782264 to 891117261
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   name              149998 non-null  object        
 1   category          150000 non-null  object        
 2   main_category     150000 non-null  object        
 3   currency          150000 non-null  object        
 4   deadline          149999 non-null  datetime64[ns]
 5   goal              150000 non-null  float64       
 6   launched          149999 non-null  datetime64[ns]
 7   pledged           150000 non-null  float64       
 8   state             150000 non-null  category      
 9   backers           150000 non-null  int64         
 10  country           150000 non-null  category      
 11  usd pledged       150000 non-null  float64       
 12  usd_pledged_real  150000 non-null  float64       
dtypes: category(2), datetime64[ns](2), float64(4), i

In [65]:
df_ks.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
ID,,,,,,,,,,,,,
872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000.0,2011-08-17 06:31:31,1145.0,canceled,24,US,1145.000000,1145.000000
1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000.0,2012-07-23 20:46:48,1851.0,failed,28,US,1851.000000,1851.000000
1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000.0,2016-07-01 21:55:54,7534.0,successful,254,US,3796.000000,7534.000000
156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000.0,2017-08-28 15:47:02,16298.0,successful,367,US,2670.000000,16298.000000
1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000.0,2016-01-25 17:37:10,1.0,failed,1,CA,0.708148,0.738225


In [66]:
df_ks.isna().sum()

name                2
category            0
main_category       0
currency            0
deadline            1
goal                0
launched            1
pledged             0
state               0
backers             0
country             0
usd pledged         0
usd_pledged_real    0
dtype: int64

In [67]:
df_ks["name"] = df_ks["name"].fillna("?")
df_ks["deadline"] = df_ks["deadline"].fillna("?")
df_ks["launched"] = df_ks["launched"].fillna("?")

### Importer les données

In [68]:
collection_kickstarter.insert_many(df_ks.to_dict('records'))

In [69]:
db_series.list_collection_names()

['youtube', 'kickstarter', '{query :{gender:"Male"}, out:"sumAge"}', 'peaky']

In [70]:
collection_kickstarter.find_one()

{'_id': ObjectId('5fc50d4aff232a381ca6b6d1'),
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': datetime.datetime(2011, 9, 16, 0, 0),
 'goal': 2000.0,
 'launched': datetime.datetime(2011, 8, 17, 6, 31, 31),
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 1  

### Récupérer les 5 projets ayant reçu le plus de promesse de dons.

In [71]:
n = 5
cursor = collection_kickstarter.find({}).limit(n).sort([("pledged", -1)])

for e in cursor:
    print(e["name"])
    print(e["pledged"])
    print()

The Everyday Backpack, Tote, and Sling
6565782.5

Pono Music - Where Your Soul Rediscovers Music
6225354.98

The Veronica Mars Movie Project
5702153.38

Zombicide: Black Plague
4079204.52

The Dash ï¿½ Wireless Smart In Ear Headphones
3390551.0



## Question 2

### Compter le nombre de projets ayant atteint leur but.

In [72]:
# successful : state

In [73]:
cursor = collection_kickstarter.find({"state":"successful"})

len(list(cursor))

53040

## Question 3

### Compter le nombre de projets pour chaque catégorie.

In [74]:
cursor = collection_kickstarter.aggregate([{"$group" : {"_id" : "$category", "counted" : {"$sum" : 1}}}])

{e["_id"]:e["counted"] for e in cursor}

{"Farmer's Markets": 175,
 'Wearables': 508,
 'Sculpture': 737,
 'Childrenswear': 192,
 'Design': 1641,
 'Live Games': 394,
 'Tabletop Games': 5581,
 'Technology': 2690,
 'Footwear': 379,
 'DIY': 480,
 'Kids': 109,
 'Calendars': 112,
 'Indie Rock': 2192,
 'Video': 147,
 'Typography': 44,
 'Crochet': 61,
 'Mixed Media': 1118,
 'Latin': 63,
 'Animals': 97,
 'Stationery': 90,
 'Video Games': 4799,
 "Children's Books": 2686,
 'Architecture': 300,
 'Interactive Design': 146,
 'Action': 282,
 'Poetry': 532,
 'Letterpress': 24,
 'Audio': 164,
 'Fantasy': 132,
 'Drinks': 995,
 'Mobile Games': 650,
 'Flight': 158,
 'Photography': 2239,
 'Performances': 414,
 'Knitting': 78,
 'Family': 130,
 'Weaving': 38,
 'Theater': 2786,
 'Conceptual Art': 393,
 'Civic Design': 130,
 'Graphic Novels': 702,
 'Pottery': 40,
 'Restaurants': 1089,
 'Hip-Hop': 1555,
 'Nonfiction': 3390,
 'Art Books': 1065,
 'Playing Cards': 963,
 'Woodworking': 433,
 'Film & Video': 4003,
 'Sound': 255,
 'Horror': 525,
 'Vegan': 2

## Question 4

### Compter le nombre de projets français ayant été instanciés avant 2016.

In [75]:
# country : FR
# launched : <2016.01.01
import datetime

before = datetime.datetime(2016, 1, 1)
cursor = collection_kickstarter.find({"country":"FR", "launched":{"$lt":before}})

len(list(cursor))

330

## Question 5

### Récupérer les projets américains ayant demandé plus de 200 000 dollars.

In [76]:
# country : US
# goal : >200000

cursor = collection_kickstarter.find({"country":"US", "goal":{"$gt":200000}})

{e["name"]:e["goal"] for e in cursor}

{'A CALL TO ADVENTURE': 287000.0,
 'Storybricks, the storytelling online RPG': 250000.0,
 'Shine On New World': 300000.0,
 'Nightclub': 3000000.0,
 'Nastaran (Wild Rose)': 250000.0,
 'Hubo - Extension Box for iPhone': 250000.0,
 'Baja ATV Park (Suspended)': 300000.0,
 'Chihuly Installation for Orlando (Pulse Nightclub)': 1000000.0,
 'Kurt Vonnegut: Unstuck in Time': 250000.0,
 'The LAKE HOPPER is a VTOL Flying Water Craft Made in America': 3000000.0,
 '"Hill" (11 For 11) (From the writer of Rudy & Hoosiers)': 1000000.0,
 'Saints of The Classroom': 250000.0,
 'Hemingwrite - A Distraction Free Digital Typewriter': 250000.0,
 'Breakfast 24/7': 1000000.0,
 'FJE REVOLT': 500000.0,
 'Austin City Limits 40 Year History Documentary Film': 400000.0,
 'Ozark Mountain Ranch': 466000.0,
 'STEM Lesson Plan - BotBrainï¿½ Educational Products': 279646.0,
 'Guitar Godz VR: A 3D Rock Music Game': 500000.0,
 "The world's first crowd funded millionaire documentary": 1000000.0,
 'Creating an online game, 

## Question 6 

### Compter le nombre de projet ayant "Sport" dans leur nom

In [77]:
import re

regex  = re.compile(r"Sport*")
cursor = collection_kickstarter.find({"name":regex})

In [78]:
sports = [e["name"] for e in cursor]
print("Nombre :", len(sports))
print("Extrait:", sports[:6])

Nombre : 330
Extrait: ['Sportswear range', 'Mount Systems for Recreation Sports & Film (GoPro) Lighting', 'E-GoBox: Revolutionary Sports Capsule Dispenser', 'Tactical Bottle Opening Hot Sauce Dispensing Spork', 'Biz Sports Weekly', 'Hugodrawings Sports Wear']
